# Combination of all important functions
This is a summary of the important functions for constructing a network and solving it for the currents.   
First off there is the Construct function which takes the paths to the input data and builds a corresponding Networkx-Graph from it.  
Then there ist the Solve class which takes a network as input and can then construct the matrices and vectors needed to solve for the currents.   
In the end there is a snippet attached which can be used to orient the edges/current-directions. It takes a incidence-like matrix as input and changes the signs according to the potential landscape.

*Stand 28.09.23*

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import norm
from scipy.optimize import curve_fit
import pandas as pd

In [2]:
def Gauss(x, mu, sig, A):
    return A  *np.exp(-0.5 * ((x-mu)/sig)**2) * 1 /(np.sqrt(2 * np.pi * sig **2))

def Load(fname, Split=True):
    a=(np.genfromtxt(fname, delimiter =" ")).flatten()    
    if Split==True:
        #Lightforge uses an high but finite value (approx 300) to handle hopping onto already taken sites
        low=a[np.abs(a)<300]
        return low
    else:
        return a

def FitGauss(data):
    bin_heights, bin_borders, _ = plt.hist(data, bins='auto')
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
    popt, _ = curve_fit(Gauss, bin_centers, bin_heights, p0=[0.01, 0.009, len(data)/5])
    plt.close()
    return popt, bin_borders

In [3]:
WDir="/home/oliver/notebooks/medDMR/"
#WDir="./medDMR/"
def Construct(Coo=WDir+"results/material/device_data/coord_0.dat",#
              Typ=WDir+"results/material/device_data/mol_types_0.dat",#
              Ener=WDir+"results/material/device_data/site_energies_0.dat",#
              CoulombHoles=WDir+"rates/1_100000_holes_dE_coulomb.dat",#
              CoulombElectrons=WDir+"rates/1_100000_electrons_dE_coulomb.dat",#
              InjectionType="h",# for electrode simulation there are otherwise HOMO and LUMO in the same file
              J=0.003, T=300, Lam=0.2):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    if InjectionType=="h":
        energies = np.genfromtxt(Ener, delimiter=" ")[:,0]
    elif InjectionType=="e":
        energies = np.genfromtxt(Ener, delimiter=" ")[:,1]
    else:
        energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1.9))
    
    #Calculate Distribution for Coulombenergydifferences
    ele=Load(CoulombElectrons)
    hol=Load(CoulombHoles)
    data=np.concatenate((ele, hol))
    popt, bin_borders=FitGauss(data)
    
    ## Inizializing Marcus Rates ##
    count=0  
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante 
        #from settings file
        #J = attempt frequency (largest possible rate) =>  "maximum ti"
        #Lam =  Reorganization Energy = lambda = materials => l
        kbT=0.0000861801*T #eV
        H_Bar = 6.58264*(10**(-16)) # Planck in eV
        Qe = 1.602176634*(10**(-19)) # Electron charge
        
        ## v is start and u is end
        deltaE=F.nodes[u]["potential"] - F.nodes[v]["potential"] + np.random.default_rng().normal(popt[0], popt[1]) 
        rate=2 * np.pi / H_Bar * np.abs(J)**2 * np.sqrt(1/(4 * Lam * kbT)) * np.exp(np.longdouble(-((Lam + deltaE)**2)/(4* Lam * kbT)))

        resistance =  deltaE / (Qe * rate )
        
        if deltaE <0: ## flow from high potential to low
            F.edges[u,v]['weight'] = resistance
        else: ## reorient edge
            # Could not just delete edge and add oposite as (Non-Di-)Graphs sort Edges by the sequence of nodes
            # Therefore I just add a negative sign to the resistor and thereby flipping the current direction
            F.edges[u,v]['weight'] = (-1) * resistance
    
    return F


#Gitter=Construct()

In [4]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def conductances(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(1/np.asarray(list((nx.get_edge_attributes(self.G, "weight").values()))), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (self.conductances() @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   

In [5]:
def Orientation(G):
    #Code Snippet that could be used for orienting the edges
    Mat=G.incidence()
    V=G.voltages()
    for x in range(np.shape(Mat)[0]):
        Y=[]
        for y in range(np.shape(Mat)[1]):
            if Mat[x,y] !=0:
                Y.append(y)
        Mat[x,Y[0]]=-np.sign(V[Y[0]]-V[Y[1]])
        Mat[x,Y[1]]=-Mat[x,Y[0]]
    return Mat

In [6]:
Gitter=Construct()
Setup=Solve(Gitter)
Currents=Setup.currents()
highXEnd=np.array(Gitter.edges())[:,:,0]==np.max(np.array(Gitter.nodes())[:,0])
lowXEnd=np.array(Gitter.edges())[:,:,0]==np.min(np.array(Gitter.nodes())[:,0])
highFilter=highXEnd[:,0]^highXEnd[:,1]
lowFilter=lowXEnd[:,0]^lowXEnd[:,1]
total_OUT=np.sum(Currents[lowFilter])
total_IN=np.sum(Currents[highFilter])

In [7]:
print("IN")
print(total_IN)
print("OUT")
print(total_OUT)

IN
9.8444695603732311296e-05
OUT
0.00042452665627496381433
